In [1]:
from nested_cv import NestedCV

import pandas as pd
import numpy as np
from sklearn.datasets import load_boston, load_iris, load_breast_cancer
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

# When using Random Search, we get a user warning with this little number of hyperparameters
# Suppress it
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)

# Regression Example

In [4]:
boston = load_boston()
X = boston.data
y = boston.target

# Define a parameters grid
param_grid = {
     'max_depth': [3, None],
     'n_estimators': [10]
}

NCV = NestedCV(model=RandomForestRegressor(), params_grid=param_grid, outer_kfolds=5, inner_kfolds=5,
               cv_options={'sqrt_of_score':True, 'randomized_search_iter':30,
                           'recursive_feature_elimination':True, 'rfe_n_features':2})
NCV.fit(X=X,y=y)

NCV.outer_scores

[4.823125961211136,
 3.8693442362761874,
 4.244929662826597,
 3.671864736980944,
 4.1530803730797885]

# Classification Example

## Breast Cancer (2 Classes)

In [5]:
from sklearn.metrics import roc_auc_score

# Binary classification
cancer = load_breast_cancer()
X = cancer.data
y = cancer.target

# Define a parameters grid
param_grid = {
     'max_depth': [3, None],
     'n_estimators': [10, 20]
}

NCV = NestedCV(model=RandomForestClassifier(), params_grid=param_grid, outer_kfolds=5, inner_kfolds=5,
               cv_options={'metric':roc_auc_score, 'metric_score_indicator_lower':False,
                           'randomized_search_iter':30, 'predict_proba':True})
NCV.fit(X=X,y=y)

NCV.outer_scores

[0.950231124807396,
 0.993940386505077,
 0.9987535612535613,
 0.9945750452079566,
 0.9972279972279972]

## Iris (3 Classes)

In [2]:
from sklearn.metrics import f1_score

# Multiclass classification
iris = load_iris()
X = iris.data
y = iris.target

# Define a parameters grid
param_grid = {
     'max_depth': [3, None],
     'n_estimators': [10, 20]
}

NCV = NestedCV(model=RandomForestClassifier(), params_grid=param_grid, outer_kfolds=5, inner_kfolds=5,
               cv_options={'metric':f1_score, 'metric_score_indicator_lower':False,
                           'randomized_search_iter':30, 'predict_proba':True,
                           'multiclass_average': 'micro'})
NCV.fit(X=X,y=y)

NCV.outer_scores

[[1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [0.         1.         0.        ]
 [0.         1.         0.        ]
 [0.         0.98       0.02      ]
 [0.         1.         0.        ]
 [0.         1.         0.        ]
 [0.         0.98       0.02      ]
 [0.         1.         0.        ]
 [0.         1.         0.        ]
 [0.         1.         0.        ]
 [0.         1.         0.        ]
 [0.         0.004      0.996     ]
 [0.         0.004      0.996     ]
 [0.         0.054      0.946     ]
 [0.         0.004      0.996     ]
 [0.         0.62166667 0.37833333]
 [0.         0.304      0.696     ]
 [0.         0.004      0.996     ]
 [0.         0.279      0.721     ]
 [0.         0.004      0.996     ]] (24, 3) [0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2]


ValueError: Classification metrics can't handle a mix of multiclass and continuous-multioutput targets